In [ ]:
%load_ext pycodestyle_magic

In [ ]:
### This cell is uneditable ###
import pandas as pd
import nltk, json, random
from IPython.display import JSON
from sklearn.model_selection import train_test_split

nltk.download('wordnet', quiet=True)
nltk.download("stopwords", quiet=True)

# load data:
vehicles = ["car", "plane", "tank"]
with open("planes/data.json") as IN:
    data = pd.DataFrame(json.load(IN))

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data['description'],
                                                    data['type'],
                                                    stratify=data['type'],
                                                    random_state=21)

JSON([len(x_train) + len(y_train), len(x_test) + len(y_test)])

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer

# Preprocess First
def preprocess(docs):
    # Apply some function to each document/review in the pandas series
    # NOTE: Every x document is a sentence, with multiple words
    return docs.apply(lambda x: x)

x_train = preprocess(x_train)
y_train = preprocess(y_train)

# Pick Vectorizer
vectorizer = CountVectorizer(binary=True)
x_train_features = vectorizer.fit_transform(x_train)
x_test_features = vectorizer.transform(x_test)
def train(classifier, x, y):
    classifier.fit(x, y)

# Pick Classifier
classifier = DummyClassifier(strategy="uniform")

# Train
train(classifier,x_train_features, y_train)

In [ ]:
### This cell is uneditable ###
from sklearn.metrics import f1_score
from IPython.display import JSON

y_pred_test = classifier.predict(x_test_features)
y_proba = classifier.predict_proba(x_test_features)

x_test_list = x_test.to_list()
y_test_list = y_test.to_list()
experiment_size = y_pred_test.size // 5
experiment = []
y_true = []
y_pred = []

for i in range(y_pred_test.size):
    y_true.append(y_test_list[i])
    y_pred.append(y_pred_test[i])
    experiment.append({
        "inputText": x_test_list[i],
        "realLabel": y_test_list[i],
        "classifierLabel": y_pred_test[i],
        "confidence": max(y_proba[i])
    })

JSON([experiment])